In [1]:
import sys
# make sure to use position 1
sys.path.insert(1, "./")

In [2]:
import numpy as np
import tensorflow as tf
import cv2
import time
import os, pickle
import src.module.nolbo as nolboModule
import dataset_utils.dataset_loader.Imagenet_dataset as imagenetDataset
import dataset_utils.dataset_loader.RenderforCNN_dataset as renderDataset
import dataset_utils.dataset_loader.nolbo_dataset as nolboDataset
import dataset_utils.dataset_loader.vae3D_dataset as vaeDataset

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [9]:
a = []
a.append([1,2,3])
a.append([4,5,6])
a = np.array(a)
b =np.arange(2)
np.random.shuffle(b)
print a[b]

[[4 5 6]
 [1 2 3]]


In [3]:
dataPath = '/media/yonsei/4TB_HDD/dataset/ObjectNet3D/CAD_training_data_numpy/'
dataset = vaeDataset.ObjectNet3D_voxelRotationDataset(
    trainingDataPath=dataPath,
    partitionNum=20,
    loadVoxOrg=True
)

load partition...
aeroplane
bed
bench
bicycle
boat
bookshelf
bottle
bus
cabinet
can
cap
car
chair
computer
cup
diningtable
door
fire_extinguisher
jar
keyboard
laptop
microwave
motorbike
mouse
piano
pillow
printer
refrigerator
road_pole
sofa
speaker
suitcase
teapot
toilet
train
trash_bin
tub
tvmonitor
done!


In [4]:
batchData = dataset.getNextBatch(32)

In [6]:
eulerangle = batchData['EulerAngle']
print eulerangle/180.0*np.pi

[[ 3.58513318  1.88584571  6.26341073]
 [ 3.68817742 -2.21861019  6.08142525]
 [ 5.5701834  -0.69725555  3.16785986]
 [ 4.3091132  -1.61026346  5.6234683 ]
 [ 1.08551975 -0.53386131  4.41976708]
 [ 4.25258199  1.46798247  5.76052901]
 [ 4.30572726 -0.25987953  4.90428793]
 [ 3.88697042  2.2677063   6.14902185]
 [ 3.68362211  3.10261945  0.15603785]
 [ 1.42309784 -1.07980205  5.2953464 ]
 [ 2.88673968  1.1927999   5.85226351]
 [ 2.75154647 -1.59949303  2.83823698]
 [ 5.76042429 -1.90172821  3.70001075]
 [ 1.60608339 -0.77676227  5.86030948]
 [ 0.70265559 -1.19941121  2.59256443]
 [ 6.07412977 -2.4842493   6.25070473]
 [ 2.8868793   2.78915832  4.91369526]
 [ 0.70263465 -1.0591356   3.05975416]
 [ 3.24683601  0.50409472  1.65036589]
 [ 3.96229883 -2.27252341  2.45979723]
 [ 0.4940329  -1.18079727  0.96679372]
 [ 5.20060993 -0.55898533  4.34454339]
 [ 3.31504347  0.77582329  1.0788491 ]
 [ 6.0393279   0.37970511  3.44700782]
 [ 3.48517817  2.38200791  0.94608539]
 [ 5.33030535 -0.25582164

In [3]:
nolboConfig = {
    'inputImgDim':[None, None, 3],
    'maxPoolNum':5,
    'predictorNumPerGrid':2,
    'bboxDim':5,
    'class':True, 'zClassDim':64, 'classDim':40,
    'inst':True, 'zInstDim':64, 'instDim':1000,
    'rot':True, 'zRotDim':3, 'rotDim':3,
    'trainable':True, 'learningRate':0.0001,
    'decoderStructure':{
        'outputImgDim':[64,64,64,1],
        'trainable':True,
        'filterNumList':[512,256,128,64,1],
        'kernelSizeList':[3,3,3,3,3],
        'stridesList':[1,2,2,2,2],
        'activation':tf.nn.elu,
        'lastLayerActivation':tf.nn.sigmoid
    }
}

In [4]:
dataPath_Object3D = '/media/yonsei/500GB_SSD/ObjectNet3D/'
dataPath_Pascal3D = '/media/yonsei/500GB_SSD/PASCAL3D+_release1.1/'
dataPath_pix3D = 'None'

dataset = nolboDataset.nolboDatasetSingleObject(
    nolboConfig=nolboConfig,
    mode='nolbo',
    dataPath_ObjectNet3D=dataPath_Object3D,
    dataPath_Pascal3D=dataPath_Pascal3D,
    dataPath_pix3D=dataPath_pix3D,
    loadOrg3DShape = True,
)

create dict...
ObjectNet3D...
load data path...
/media/yonsei/500GB_SSD/PASCAL3D+_release1.1/
imagenet 21857/21857
pascal 06646/06646
/media/yonsei/500GB_SSD/ObjectNet3D/
imagenet 90126/90126
done!
data path shuffle...
done! : 238882


In [5]:
dataset.setInputImageSize([224, 224, 3])

In [10]:
start = time.time()
for i in range(10):
    batchData = dataset.getNextBatchPar(32)
print time.time() - start

1.85034608841


In [11]:
start = time.time()
for i in range(10):
    batchData = dataset.getNextBatch(32)
print time.time() - start

5.47306489944


In [ ]:
a = [int(float('1.2')), int(3.1)]

In [ ]:
datasetPath = '/media/yonsei/3A1FF8D034F3B6C8/RenderforCNN/'
dataset = renderDataset.RenderforCNNDataset(dataPath=datasetPath, classNum=12, instNum=8600, rotDim=360)
dataset.setImageSize((224, 224))

In [ ]:
batchData = dataset.getNextBatchPar(batchSize=256)

In [ ]:
images = batchData['inputImages']
classList = batchData['classList']
for i in range(len(images)):
    cv2.imwrite('test/'+str(i)+'.jpg', images[i])
    print i, np.argmax(classList[i])

In [ ]:
a = []
a.append([1,2,3])
a.append(None)
print a

In [ ]:
from xml.etree.cElementTree import parse
tree = parse("test.xml")
note = tree.getroot()

In [ ]:
tree = parse("test.xml")
note = tree.getroot()
note.find('object').find('bndbox').findtext('xmin')

In [ ]:
a = '/fuck/ass/hole/'
b = a.split('ass')
print b

In [ ]:
acc = np.array([1,2,3,4,5])
sys.stdout.write("acc=c:{:.5f},i:{:.5f},a:{:.5f},e:{:.5f},ip:{:.5f}\r".format(acc[0],acc[1],acc[2],acc[3],acc[4]))

In [4]:
# datasetPath = '/media/yonsei/4TB_HDD/dataset/ImageNet/'
datasetPath = '/media/yonsei/3A1FF8D034F3B6C8/ImageNet'
dataset = imagenetDataset.imagenetDataset(dataPath=datasetPath, classNum=1000)
dataset.setImageSize((224, 224))

load data point path...
done!1000


In [8]:
start = time.time()
for i in range(1):
    batchData = dataset.getNextBatchPar(32)
print time.time() - start

0.192816972733


In [10]:
start = time.time()
for i in range(1):
    batchData = dataset.getNextBatch(32)
print time.time() - start

0.151700019836


In [12]:
print batchData.keys()

['inputImages', 'classIndexList']


In [ ]:
images = batchData['inputImages']
classList = batchData['classIndexList']
for i in range(len(images)):
    cv2.imwrite('test/'+str(i)+'.jpg', images[i])
    print i, np.argmax(classList[i])

In [ ]:
nolboConfig = {
    'inputImgDim':[416,416,3],
    'maxPoolNum':5,
    'predictorNumPerGrid':2,
    'bboxDim':5,
    'class':True, 'zClassDim':64, 'classDim':101,
    'inst':True, 'zInstDim':64, 'instDim':1000,
    'rot':True, 'zRotDim':3, 'rotDim':3,
    'trainable':True,
    'decoderStructure':{
        'outputImgDim':[64,64,64,1],
        'trainable':True,    
        'filterNumList':[512,256,128,64,1],
        'kernelSizeList':[4,4,4,4,4],
        'stridesList':[1,2,2,2,2],
        'activation':tf.nn.leaky_relu,
        'lastLayerActivation':tf.nn.sigmoid
    }
}

In [ ]:
dataPath_Object3D = '/media/yonsei/4TB_HDD/dataset/ObjectNet3D/'
dataPath_Pascal3D = '/media/yonsei/4TB_HDD/dataset/PASCAL3D+_release1.1/'
dataPath_pix3D = '/media/yonsei/4TB_HDD/dataset/pix3d/'

In [ ]:
dataset = nolboDataset.nolboDatasetSingleObject(
    nolboConfig=nolboConfig, 
    dataPath_ObjectNet3D=dataPath_Object3D,
    dataPath_Pascal3D=dataPath_Pascal3D,
    dataPath_pix3D=dataPath_pix3D
)

In [ ]:
startTime = time.time()
for i in range(3):
    batchData = dataset.getNextBatch(batchSize=32)
    batchData['learningRate'] = 0.0001
    inputImages = batchData['inputImages']
    for j in range(len(inputImages)):
        cv2.imwrite('test/'+str(i)+'_'+str(j)+'.jpg', inputImages[j])
        print str(i)+'_'+str(j), np.argmax(batchData['classList'][j])+1
print time.time() - startTime

In [ ]:
print np.argmax((1,0))

In [ ]:
print batchData.keys()
inputImages = batchData['inputImages']
# outputImages = batchData['outputImages']
for i in range(len(inputImages)):
    cv2.imwrite('test/'+str(i)+'.jpg', inputImages[i])
    print i, np.argmax(batchData['classList'][i])+1
#     np.savetxt('test/'+str(i)+'.txt', np.reshape(outputImages[i], (64*64, 64*1)))

In [ ]:
max_idx = 0
for key in dataset._instDict_pix3D.keys():
    print key
    for subKey in dataset._instDict_pix3D[key].keys():
        if max_idx<dataset._instDict_pix3D[key][subKey]:
            max_idx = dataset._instDict_pix3D[key][subKey]
            print subKey, dataset._instDict_pix3D[key][subKey]

In [ ]:
batchData = dataset.getNextBatch(batchSize=128, outputOrg=False)

In [ ]:
print batchData.keys()

In [ ]:
for i in range(len(batchData['classList'])):
    print i, np.argmax(batchData['classList'][i]), np.argmax(batchData['instList'][i])

In [ ]:
inputImages = batchData['inputImages']
outputImages = batchData['outputImages']
for i in range(len(inputImages)):
    cv2.imwrite('test/'+str(i)+'.jpg', 255.0*inputImages[i])
    np.savetxt('test/'+str(i)+'.txt', np.reshape(outputImages[i], (64*64, 64*1)))

In [ ]:
print batchData['EulerAngle'][64]
print batchData['EulerAngle'][54]
print batchData['EulerAngle'][55]

In [ ]:
classifier = nolboModule.darknet_classifier(
    batchSize=36, imgSize=(416,416), 
    coreLayerActivation = tf.nn.elu,
    lastLayerActivation=tf.nn.sigmoid,
    dataPath='./data/classifier_training_data/')

In [ ]:
classifier.train(epoch=10, weightSavePath='./weights/classifier/')

In [ ]:
classifier.saveNetworks(savePath='./weights/classifier/')   